In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [3]:
import datetime
import random
from functools import partial
from multiprocessing.pool import Pool

import Levenshtein
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm_notebook as tqdm

from src.text_transformer import PhoneticTextTransformer, TfidfTextTransformer, WmdTextTransformer
from src.corpora import load_phonetized_normalized_corpora

from data import load_questions_data
from typo_making import variants

%matplotlib inline

In [4]:
questions = load_questions_data()

In [5]:
questions[0]

['как долго рассматривается проект ',
 'я за неделю успею получить решение',
 'сколько рассматривается сделка по спецтехнике?',
 'как скоро я получу ответ по заявке по спецтехнике?',
 'сколько времени ждать по спецтехнике?',
 'сколько времени занимает рассмотрение ',
 'сколько дней принятие решения',
 'как долго уйдет на принятие решения по мне',
 'сколько времени займет принятие решения по проекту',
 'как быстро я смогу получить грузовой автомобиль',
 'Я хочу заключить лизинговую сделку завтра / сегодня ']

In [6]:
len(questions)

281

In [7]:
corpora = load_phonetized_normalized_corpora()

In [8]:
len(corpora)

102702

In [9]:
# def test_one(transformer_class, questions_groups, question):
#     test_questions_groups = []
#     group_where_is_question = None
#     for group in questions_groups:
#         if question in group:
#             if group_where_is_question is not None:
#                 print('warning: question "{}" is in both group "{}" and "{}"'.format(question, group_where_is_question, group))
#                 print('ignore question')
#                 return None
#             group_where_is_question = [it for it in group if it != question]
#             test_questions_groups.append(group_where_is_question)
#         else:
#             test_questions_groups.append(group)
            
#     assert group_where_is_question is not None
  
#     transformer = transformer_class()
#     transformer.fit(test_questions_groups)
    
#     question_vec = transformer.transform(question)
#     vecs_groups = [(transformer.transform(q), group) for group in test_questions_groups for q in group]
    
#     pred_group_where_is_question = max(vecs_groups, key=lambda it: cosine_similarity(question_vec, it[0])[0])[1]
#     return group_where_is_question == pred_group_where_is_question


# def test_one_job(p):
#     print('Start', p[0])
#     r = test_one(p[2], p[3], p[1])
#     print('✓ Done', r, p[0])
#     return r
            

# def test_all(transformer_class, questions_groups):
#     pool = Pool(4)
#     scores = pool.map(test_one_job, [(i, it, transformer_class, questions_groups) 
#                                      for i, it in enumerate([it for group in questions_groups for it in group])])
#     scores = [it for it in scores if it is not None]
    
#     accuracy = sum(scores) / len(scores)
#     return accuracy

In [10]:
def exclude_repeates_in_questions(questions_groups):
    new_groups = list()
    question_set = set()
    for group in questions_groups:
        new_groups.append([q for q in group if q not in question_set])
        for q in group:
            question_set.add(q)
    return new_groups
            

def test_one(question_group_tuple, transformer, questions_groups):
    question_vec = transformer.transform(question_group_tuple[0])
    vecs_groups = [(transformer.transform(q), group) for group in questions_groups 
                                                     for q in group
                                                     if q != question_group_tuple[0]]
    
    pred_group_where_is_question = max(vecs_groups, key=lambda it: cosine_similarity(question_vec, it[0])[0])[1]
    r = question_group_tuple[1] == pred_group_where_is_question
    print('\n', datetime.datetime.utcnow(), 'Done', r, question_group_tuple[0], '\n\ttrue:', question_group_tuple[1], '\n\tpred:', pred_group_where_is_question)
    return r
    
    
def test_all(transformer, questions_groups):
    questions_groups = exclude_repeates_in_questions(questions_groups)
    
    print(datetime.datetime.utcnow(), 'Fitting questions')
    transformer.fit(questions_groups)
    print(datetime.datetime.utcnow(), 'Fitted')
    
    pool = Pool(4)
    data = [(q, group) for group in questions_groups for q in group]
    scores = pool.map(partial(test_one, transformer=transformer, questions_groups=questions_groups), data)
    return sum(scores) / len(scores)

In [11]:
qs = random.sample([it for group in questions for it in group], 50)

In [20]:
t = PhoneticTextTransformer(corpora)
t.fit(questions)

In [21]:
from src.transforms.phonetize import phonetize

In [22]:
phonetize('нолевой'), phonetize('нулевой')

('наливай', 'нуливай')

In [23]:
for q in qs:
    print(q)
    print(t.transform(q))
    print()

кто отвечает в случае непоставки
кта атвичат ф случай нипаставка

б/у танспортные средства возможно приобрести с субсидией Минпромторга
п у танспартнай срицтва вазмажна приабристи с субсидиа минпрамтарк

хантер в списке субсидий
хантирит ф списак субсидиа

Меня выгнали из гильдии. Могу я вступить в нее снова?
а вагнат ис гилдиа мач а вступит ф ниа снава

Что находится в Сундуке Фортуны?
чта нахадитса ф сундQ фартуна

Какой максимальный уровень Злости на Слейве?
какай максималнай уравин зласт на слийф

когда заканчивается лизинг?
какда заканчиватса лизинк

Как получить фиолетовые доспехи?
как палучит фиалитавай даспY

какой порядок досрочного выкупа?
какай парадак дасрачнай вакуп

досрочное погошение возможно
дасрачнай пагашини вазмажна

можно приобрести Хендай?
мажна приабристи хиндат

Что такое личная активность в Гильдии?
чта такай личнай активнаст ф гилдиа

rio хочу взять с субсидией минпромторга
риа хатит фзат с субсидиа минпрамтарк

Можно ли получить специалистов в разных версиях 

In [16]:
# questions_subset = random.sample(questions, 25)
# print('questions count:', len([it for group in questions_subset for it in group]))
# test_all(transformer=TfidfTextTransformer(corpora=corpora), questions_groups=questions_subset)